In [11]:
import pandas as pd
from datetime import datetime,date, time
import requests
import json

In [2]:
import pyspark
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [71]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
spark = SparkSession.builder.appName("bitcoin_pyspark_kraken_json").getOrCreate()
spark

In [73]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1440&since=unix_now').json()

In [74]:
resp.keys()

dict_keys(['error', 'result'])

In [75]:
resp['result'].keys()

dict_keys(['XXBTZUSD', 'last'])

In [76]:
len(resp['result']['XXBTZUSD'])

720

In [77]:
resp['result']['XXBTZUSD'][0]

[1603411200,
 '12985.4',
 '13032.8',
 '12735.0',
 '12939.0',
 '12911.7',
 '3897.16955007',
 16226]

In [78]:
resp['result']['XXBTZUSD'][719]

[1665532800,
 '19050.0',
 '19190.0',
 '18937.3',
 '19107.9',
 '19087.9',
 '1631.92126763',
 7232]

In [79]:
btc_json = resp['result']['XXBTZUSD']

In [80]:
btc_spark_df = spark.createDataFrame(btc_json)

In [81]:
btc_spark_df.show(5)

+----------+-------+-------+-------+-------+-------+-------------+-----+
|        _1|     _2|     _3|     _4|     _5|     _6|           _7|   _8|
+----------+-------+-------+-------+-------+-------+-------------+-----+
|1603411200|12985.4|13032.8|12735.0|12939.0|12911.7|3897.16955007|16226|
|1603497600|12939.0|13172.3|12890.7|13130.9|13041.5|1813.58698169| 8277|
|1603584000|13130.9|13351.0|12833.0|13039.2|13080.1|3508.36779755|12947|
|1603670400|13039.8|13243.6|12801.0|13080.0|13038.8|4628.57769014|17227|
|1603756800|13080.0|13794.8|13064.0|13649.6|13487.3|7598.84407927|26506|
+----------+-------+-------+-------+-------+-------+-------------+-----+
only showing top 5 rows



In [82]:
# kraken_ohlc_cols = ["date","open", "high", "low", "close", "vwap","volume", "trades"]

In [83]:
btc_spark_df = btc_spark_df.withColumnRenamed('_1', "date")\
                           .withColumnRenamed('_2', "open")\
                           .withColumnRenamed('_3', "high")\
                           .withColumnRenamed('_4', "low")\
                           .withColumnRenamed('_5', "close")\
                           .withColumnRenamed('_6', "vwap")\
                           .withColumnRenamed('_7', "volume")\
                           .withColumnRenamed('_8', "trades")

In [84]:
btc_spark_df.show(5)

+----------+-------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|   vwap|       volume|trades|
+----------+-------+-------+-------+-------+-------+-------------+------+
|1603411200|12985.4|13032.8|12735.0|12939.0|12911.7|3897.16955007| 16226|
|1603497600|12939.0|13172.3|12890.7|13130.9|13041.5|1813.58698169|  8277|
|1603584000|13130.9|13351.0|12833.0|13039.2|13080.1|3508.36779755| 12947|
|1603670400|13039.8|13243.6|12801.0|13080.0|13038.8|4628.57769014| 17227|
|1603756800|13080.0|13794.8|13064.0|13649.6|13487.3|7598.84407927| 26506|
+----------+-------+-------+-------+-------+-------+-------------+------+
only showing top 5 rows



In [85]:
import pyspark.sql.functions as F

In [86]:
#btc_spark_df.withColumn("date", F.from_unixtime(F.col("date"), 'yyyy-MM-dd').cast("date")).show(5)

In [87]:
#btc_spark_df.withColumn("date", F.from_unixtime(F.col("date"), 'yyyy-MM-dd')).show(5)

In [88]:
btc_spark_df = btc_spark_df.withColumn("date", F.from_unixtime(F.col("date"), 'yyyy-MM-dd').cast("date"))

In [89]:
btc_spark_df.show(5)

+----------+-------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|   vwap|       volume|trades|
+----------+-------+-------+-------+-------+-------+-------------+------+
|2020-10-22|12985.4|13032.8|12735.0|12939.0|12911.7|3897.16955007| 16226|
|2020-10-23|12939.0|13172.3|12890.7|13130.9|13041.5|1813.58698169|  8277|
|2020-10-24|13130.9|13351.0|12833.0|13039.2|13080.1|3508.36779755| 12947|
|2020-10-25|13039.8|13243.6|12801.0|13080.0|13038.8|4628.57769014| 17227|
|2020-10-26|13080.0|13794.8|13064.0|13649.6|13487.3|7598.84407927| 26506|
+----------+-------+-------+-------+-------+-------+-------------+------+
only showing top 5 rows



In [90]:
btc_spark_df.orderBy(btc_spark_df.date.desc()).show(5)

+----------+-------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|   vwap|       volume|trades|
+----------+-------+-------+-------+-------+-------+-------------+------+
|2022-10-11|19050.0|19190.0|18937.3|19107.9|19087.9|1631.92126763|  7232|
|2022-10-10|19132.1|19259.4|18860.0|19050.0|19062.4|3503.57513226| 15956|
|2022-10-09|19443.5|19520.0|19040.0|19132.1|19291.0|2623.75289814| 15727|
|2022-10-08|19420.0|19559.5|19335.0|19443.5|19470.7| 903.92350217|  9334|
|2022-10-07|19532.8|19615.0|19249.7|19417.5|19471.7|1337.85077668| 10018|
+----------+-------+-------+-------+-------+-------+-------------+------+
only showing top 5 rows



In [91]:
spark.stop()